## Simple agent the that randomly chooses allowable actions within a silhouette ('viable' actions)

Programming this agent as a baseline.
If we can bias the agent towards e.g. bigger blocks, that's a plus.

### input
Bitmap representation of each stimulus

### output
Output block placements as dataframe of same type as action dataframe used in other analyses (dfi) i.e.
targetName, blockNum, x,y,w,h

### stability
Blocks have to be placed on a 'floor', which includes two separated floors (to make a henge) 

In the experiment, unstable placements end the trial. We could:
a) allow the agent to make unstable placements, but end the trial when they do
b) not allow the agent to consider unstable placements
Here I go for option b, where possible actions don't include those that would fall

### action selection 
There are various ways to make a random agent:
a) enumerate all possible actions (all blocks in all locations), then uniformly select from these.
b) uniformly select a block, then uniformly select a location.
c) uniformly select a location, then uniformly select a block that fits there. 
d) uniformly select a block **and** uniformly select a location, reject if not possible.



In [1]:
from __future__ import division

import numpy as np
import os, sys
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
proj_dir = os.path.abspath('../..')

from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

import sklearn.metrics as metrics

from IPython.core.pylabtools import figsize, getfigs

import seaborn as sns

import random

from scipy.stats import norm
from IPython.display import clear_output

import numpy as np
import pandas as pd
import os
import json

import copy
import importlib

### Add Paths

## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'..','..')) ## use relative paths

## add helpers to python path
import sys
if os.path.join(proj_dir, 'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'stimuli'))

## custom helper modules
import separation_axis_theorem as sat
#import blockworld_helpers as utils
#import display_world as stability #may want to make a separate module for stability

def cls():
    os.system('cls' if os.name=='nt' else 'clear')

import scoring
    

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)   

In [3]:
#### Target maps: grab the bitmap representation of each stim

targets = ['hand_selected_004', 'hand_selected_005', 'hand_selected_006',
       'hand_selected_008', 'hand_selected_009', 'hand_selected_011',
       'hand_selected_012', 'hand_selected_016']

target_maps = {}

with open(os.path.abspath('../results/csv/targetMaps.txt')) as json_file:
    target_maps = json.load(json_file)
    

def check_overlap(x,y,w,h,world, mode='inside'):
    overlaps = False
    
    if mode == 'inside':
        overlaps = np.all(world[x:(x+w),y:(y+h)])
    elif mode == 'outside':
        overlaps = ~np.any(world[x:(x+w),y:(y+h)])
    else:
        return
    
    return overlaps

def check_stability(x,y,w,h,world):
    '''
    checks to see if block would be supported without falling using heuristics.
    Does not allow side-supported blocks, which are sometimes possible in the real experiments
    '''
    
#     if ((w==4) & (y==2) & (x==8)):
#         print(np.rot90(world.astype(int)))
    
    if y == 0: #if on the floor then will be stable
        return True
    else: #if greater than 1/2 of the block is supported then stable
        support = world[x:(x+w),y-1:y].astype(int)
        if np.sum(support) > w/2:
            return True
        # supports on both sides of long block
        elif (w == 4):
            left_sum = sum(world[x:(x+2),y-1:y].astype(int))
            right_sum = sum(world[x+2:(x+w),y-1:y].astype(int))
            if ((left_sum>= 1) & (right_sum >= 1)):                
                return True
            else: return False
        else:
              return False

def find_positions(world, block, x_offset = 5):
    
    positions = []
    
    for i in range(world.shape[0]-block['width']+1):
            if (~np.any(world[i:i+block['width'],0])):
                positions.append({'x': i + x_offset,
                                  'y': 0})
                
    for j in range(1,world.shape[1]-block['height']+1):
        for i in range(world.shape[0]-block['width']+1):
            if ((~np.any(world[i:i+block['width'],j])) & np.any(world[i:i+block['width'],j-1])):
                positions.append({'x': i + x_offset,
                                  'y': j})
    return positions
                
    

def simulate(targets, niter, verbose = False, provide_actual_target=False):
    
    block_dims = [{'width':1,
                   'height':2},
                  {'width':2,
                   'height':1},
                  {'width':2,
                   'height':2},
                  {'width':2,
                   'height':4},
                  {'width':4,
                   'height':2}]
    block_dims.reverse()

    world_bounds = {'left': 5,
                    'right': 13}
    
    columns = ['targetName','run','blockNum','discreteWorld','perfect','x','y','w','h']

    df = pd.DataFrame(columns=columns)
    
    for target in targets:

        if provide_actual_target:
            target_map = target
        else:
            target_map = np.logical_not(np.array(target_maps[target]))

        for run in range(0,niter):
            
            discrete_world = np.zeros([18,13]).astype(bool)

            block_num = 0
            completed = False
            tested_all_blocks = False

            while (~completed & ~tested_all_blocks):

                placed = False

                random.shuffle(block_dims)

                b = 0
                while((b < len(block_dims)) & ~placed): #keep trying blocks until placed or none left

                    #select next block from shuffled list
                    block = block_dims[b]
                    if verbose: print(" "*0,'block:', block)

                    # position-centric
                    # enumerate all positions for that block
                    positions = find_positions(discrete_world[world_bounds['left']:world_bounds['right'],0:8], block, x_offset=5)
                    if verbose: print(positions)

                    random.shuffle(positions) # shuffle positions
                    p = 0

                    while(~placed & (p < len(positions))): #keep trying positions until placed or none left
                        position = positions[p]
                        if verbose: print(" "*4,'position:', position)

                        x_loc = position['x']
                        y_loc = position['y']

                        # check if valid location
                        # check if in silhouette
                        within_silhouette = check_overlap(x_loc,y_loc,block['width'],block['height'], target_map, mode = 'inside')
                        if verbose: print(" "*4,'within silhouette:', within_silhouette)

                        if within_silhouette:
                             # check if free in current world
                            free_space = check_overlap(x_loc,y_loc,block['width'],block['height'], discrete_world, mode = 'outside')
                            if verbose: print(" "*5,'free space:', free_space)

                            if free_space:

                                # check stability
                                stable = check_stability(x_loc, y_loc, block['width'], block['height'], discrete_world)
                                if verbose: print(" "*6,'stable:', stable)

                                #if added:
                                if stable:
                                    # add to world
                                    discrete_world[x_loc:x_loc+block['width'],y_loc:y_loc+block['height']] = 1
                                    completed = np.all(np.equal(discrete_world,target_map))
                                    df = df.append({'targetName': str(target),
                                                   'run': run,
                                                   'blockNum': block_num,
                                                   'discreteWorld':discrete_world.copy(),
                                                   'perfect':completed,
                                                   'x':x_loc,
                                                   'y':y_loc,
                                                   'w':block['width'],
                                                   'h':block['height']}, ignore_index=True)
                                    if verbose: print(np.rot90(discrete_world.astype(int)))
                                    placed = True
                                    
                                    if (completed & verbose):
                                        print('completed structure!')
                                    block_num += 1
                                else:
                                    p += 1 # check next position
                            else:
                                p += 1 # check next position
                        else:
                            p += 1 # check next position

                    if(p == len(positions)): # if no positions work
                        b += 1 # check next block

                if b == len(block_dims):
                    if verbose: print('no viable blocks- giving up')
                    tested_all_blocks = True
                    
                    
    return df

In [285]:
test_target = np.zeros([18,13]).astype(bool)
#test_target[5:9,0:2] = 1
test_target[5:13,0:8] = 1
#test_target[5:9,2:4] = 1
print(np.rot90(test_target).astype(int))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]]


In [277]:
print(np.rot90(target_maps[targets[0]]).astype(int))

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 1]]


In [315]:
df_try = simulate([test_target],1,verbose=True, provide_actual_target=True)

 block: {'height': 2, 'width': 1}
[{'x': 5, 'y': 0}, {'x': 6, 'y': 0}, {'x': 7, 'y': 0}, {'x': 8, 'y': 0}, {'x': 9, 'y': 0}, {'x': 10, 'y': 0}, {'x': 11, 'y': 0}, {'x': 12, 'y': 0}]
     position: {'x': 10, 'y': 0}
     within silhouette: True
      free space: True
       stable: True
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]]
 block: {'height': 2, 'width': 4}
[{'x': 5, 'y': 0}, {'x': 6, 'y': 0}, {'x': 7, 'y': 2}, {'x': 8, 'y': 2}, {'x': 9, 'y': 2}]
     position: {'x': 6, 'y': 0}
     within silhouette: True
      free space: T

In [ ]:
df_try = simulate([targets[0]],1,verbose=True)

## Run simulations

In [ ]:
df_random_agent = simulate(targets,1000,verbose=False)

In [ ]:
df_random_agent

In [290]:
out_path = os.path.join(csv_dir,'block_silhouette_initial_random_agent.csv')
df_random_agent.to_csv(out_path)

In [291]:
import ast

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import plotly
import plotly.graph_objects as go
import plotly.io as pio
pio.orca.config.use_xvfb = True
plotly.io.orca.config.save()

import importlib
import trajectory as g

In [293]:
#df_random_agent['discreteWorld']
df_random_agent['rawF1DiscreteScore'] = df_random_agent.apply(scoring.get_f1_score_lambda, axis = 1)
df_random_agent['discreteWorld'] = df_random_agent['discreteWorld'].apply(lambda a: a*1)

In [294]:
df_random_agent

,targetName,run,blockNum,discreteWorld,perfect,x,y,w,h,rawF1DiscreteScore
0,hand_selected_004,0,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,7,0,2,4,0.266667
1,hand_selected_004,0,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,7,4,1,2,0.322581
2,hand_selected_004,0,2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,10,0,2,2,0.424242
3,hand_selected_004,0,3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,10,2,2,2,0.514286
4,hand_selected_004,0,4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,10,4,2,1,0.555556
5,hand_selected_004,0,5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,9,5,4,2,0.700000
6,hand_selected_004,0,6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,5,0,2,2,0.761905
7,hand_selected_004,0,7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,7,6,1,2,0.790698
8,hand_selected_004,0,8,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,11,7,2,1,0.818182
9,hand_selected_004,0,9,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",False,6,2,1,2,0.844444


In [ ]:
targets = ['hand_selected_004', 'hand_selected_005', 'hand_selected_006',
       'hand_selected_008', 'hand_selected_009', 'hand_selected_011',
       'hand_selected_012', 'hand_selected_016']

df_random_agent['gameID'] = df_random_agent['run']
df_random_agent['phase_extended'] = 'random agent'
df_random_agent['flatDiscreteWorld'] = df_random_agent['discreteWorld'].apply(lambda a: (1+(-1)*np.array(a)).flatten())

importlib.reload(g) ## reimport graph utils
make_plot = True
if make_plot:
    phases = ['simulation']
    for this_target in targets:
        for this_phase in phases:
            g.plot_trajectory_graph(data = df_random_agent, 
                                    target_name = this_target, 
                                    phase = this_phase, 
                                    save = False, 
                                    out_dir = plot_dir,
                                    extension = 'test',
                                    x_lower_bound = 4,
                                    x_upper_bound = 13,
                                    edge_width_scale_factor = 0.4,
                                    node_size_scale_factor = 0.4)

## Load csvs from multithreaded agent

In [23]:
import glob

in_path = os.path.join(csv_dir,'agent_results/')

all_files = glob.glob(in_path + "/*100_thread_*.csv")
[print(file) for file in all_files]
li = []

for filename in all_files:
    batch = pd.read_csv(filename, index_col=None, header=0)
    li.append(batch)

df = pd.concat(li, axis=0, ignore_index=True)


/home/wmccarth/block_construction/analysis/results/csv/agent_results/block_silhouette_initial_random_agent_100_thread_2.csv
/home/wmccarth/block_construction/analysis/results/csv/agent_results/block_silhouette_initial_random_agent_100_thread_5.csv
/home/wmccarth/block_construction/analysis/results/csv/agent_results/block_silhouette_initial_random_agent_100_thread_6.csv
/home/wmccarth/block_construction/analysis/results/csv/agent_results/block_silhouette_initial_random_agent_100_thread_4.csv
/home/wmccarth/block_construction/analysis/results/csv/agent_results/block_silhouette_initial_random_agent_100_thread_1.csv
/home/wmccarth/block_construction/analysis/results/csv/agent_results/block_silhouette_initial_random_agent_100_thread_7.csv
/home/wmccarth/block_construction/analysis/results/csv/agent_results/block_silhouette_initial_random_agent_100_thread_3.csv


[None, None, None, None, None, None, None]

In [21]:
df.groupby('targetName')['runID'].nunique()

targetName
hand_selected_004    700
hand_selected_005    700
hand_selected_006    700
hand_selected_008    700
hand_selected_009    700
hand_selected_011    700
hand_selected_012    700
hand_selected_016    700
Name: runID, dtype: int64

In [22]:
df

,Unnamed: 0,targetName,run,runID,blockNum,discreteWorld,perfect,x,y,w,h
0,0,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,0,[[False False False False False False False Fa...,False,7,0,1,2
1,1,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,1,[[False False False False False False False Fa...,False,5,0,1,2
2,2,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,2,[[False False False False False False False Fa...,False,10,0,2,1
3,3,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,3,[[False False False False False False False Fa...,False,6,0,1,2
4,4,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,4,[[False False False False False False False Fa...,False,10,1,2,4
5,5,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,5,[[False False False False False False False Fa...,False,6,2,2,1
6,6,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,6,[[False False False False False False False Fa...,False,8,0,1,2
7,7,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,7,[[False False False False False False False Fa...,False,10,5,2,2
8,8,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,8,[[False False False False False False False Fa...,False,6,3,2,2
9,9,hand_selected_004,0,54fb30b4-9887-11ea-ae55-9f44a999382c,9,[[False False False False False False False Fa...,False,7,5,1,2


In [16]:
! git push

Counting objects: 9, done.
Delta compression using up to 16 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 5.18 KiB | 52.00 KiB/s, done.
Total 9 (delta 8), reused 0 (delta 0)
remote: Resolving deltas: 100% (8/8), completed with 8 local objects.
To github.com:cogtoolslab/block_construction.git
   217b118..88ac3c0  master -> master
